In [26]:
!pip install sentence-transformers faiss-cpu

!pip install matplotlib scikit-learn
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 75.9 MB/s eta 0:00:00


In [27]:
from google.colab import drive
drive.mount('/content/drive')

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# 1. 임베딩 모델 로드
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 2. 학습 데이터 로딩
df = pd.read_csv('/content/drive/MyDrive/combined_dataset.csv')
originals = df["original_text"].tolist()
simples = df["simple_text"].tolist()

# 3. 문장 임베딩
original_embeddings = model.encode(originals, convert_to_numpy=True, show_progress_bar=True)

# 4. FAISS 인덱스 구축 (cosine similarity 기반)
index = faiss.IndexFlatIP(original_embeddings.shape[1])
faiss.normalize_L2(original_embeddings)
index.add(original_embeddings)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [28]:
# SentenceEmbedding + FAISS Retrieval + Copy-based Rewrite

def find_nearest(original_input, top_k=1):
    # 입력 임베딩
    embedding = model.encode([original_input], convert_to_numpy=True)
    faiss.normalize_L2(embedding)

    # 최근접 검색
    distances, indices = index.search(embedding, top_k)

    # 결과 반환
    nearest_simple = [simples[i] for i in indices[0]]
    return nearest_simple[0]


In [34]:
test_sentences = [
    # 출처 1: 서울성모병원
    "전신 마취 후 당일 은 심호흡을 자주 합니다.",
    "경미한 오심, 구토 증상이 있을 수 있으며 차츰 호전됩니다.",
    "수술당일은 안정을 취하며 무리한 운동은 1주정도 피하도록 합니다.",
    "수술 후 당일 자가 운전 금기입니다.",
    "수술 부위 상처가 치유 될 때 까지 음주는 피합니다.",
    "수술 부위 통증이 심한 경우 처방 받은 진통제를 복용 하도록 합니다.",
    "수술부위 출혈, 부종, 가라 앉지 않는 통증 시 병원으로 문의 하도록 합니다.",

    # 출처 2: 전북대병원
    "수술 부위는 48시간동안 얼음찜질 하 십시오.",
    "음식은 1주일 정도 유동식으로 드십시오.",
    "덩어리 있는 것, 매운 것, 뜨거운 것, 자극적인 것은 48시간 동안 드시지 마십시오.",
    "수술 후 3일째(48시간 이후)부터는 온 찜질을 하시고, 미지근한 음식을 드셔도 됩니다.",
    "식사 후에는 깨끗한 물이나 구강청정 제(처방된 소독약)를 이용하여 수술부 위를 청결하게 관리하십시오.",
    "수술 후 당일은 무리한 일은 피하시고 휴식을 취하십시오.",
    "뜨거운 물에 목욕을 하거나, 땀흘리는 운동을 하실 경우 수술부위가 덧날 수 있으니 1주일 정도는 피하시고, 가벼 운 샤워정도는 하셔도 됩니다.",
    "음주와 흡연은 수술 후 급성출혈, 감 염, 염증, 급성동통, 수술부위 치유지 연, 수술부위의 괴사가 발생할 가능성 이 아주 높으므로 열흘정도(적어도1주 일)는 자제하셔야 합니다.",
    "수술한 부위를 보기 위해서 입술을 심 하게 들추거나 잇솔질을 하시면 수술 부위의 살이 찢어질 수 있으니 그러한 행동은 피하십시오.",
    "특별한 이상이 발생된 경우, 연락을 하도록 하십시오."
]


for sentence in test_sentences:
    rewrite = find_nearest(sentence)
    print("테스트 문장:", sentence)
    print("리라이팅:", rewrite)
    print("────────────────────────────")


테스트 문장: 전신 마취 후 당일 은 심호흡을 자주 합니다.
리라이팅: 너무 안 움직이면 아플 수 있어요. 그래서 조금씩 자주 움직이고, 아픈 데 주위는 살살 문질러 주세요.
────────────────────────────
테스트 문장: 경미한 오심, 구토 증상이 있을 수 있으며 차츰 호전됩니다.
리라이팅: 수술한 데가 좀 붓거나 피부 느낌이 이상할 수 있는데, 보통은 잠깐만 그래요.
────────────────────────────
테스트 문장: 수술당일은 안정을 취하며 무리한 운동은 1주정도 피하도록 합니다.
리라이팅: 수술한 지 한 달 동안은 힘든 운동은 하지 마세요.
────────────────────────────
테스트 문장: 수술 후 당일 자가 운전 금기입니다.
리라이팅: 수면내시경을 한 날은 졸릴 수 있어 운전하면 안 돼요.
────────────────────────────
테스트 문장: 수술 부위 상처가 치유 될 때 까지 음주는 피합니다.
리라이팅: 상처가 다 낫기 전까지는 목욕하지 마세요.
────────────────────────────
테스트 문장: 수술 부위 통증이 심한 경우 처방 받은 진통제를 복용 하도록 합니다.
리라이팅: 아플 때는 병원에서 받은 약을 알려준 대로 잘 챙겨 드세요.
────────────────────────────
테스트 문장: 수술부위 출혈, 부종, 가라 앉지 않는 통증 시 병원으로 문의 하도록 합니다.
리라이팅: 열이 나거나 상처가 많이 아프고 붓고 피가 나면 병원에 연락하세요.
────────────────────────────
테스트 문장: 수술 부위는 48시간동안 얼음찜질 하 십시오.
리라이팅: 관절 검사를 한 부위는 반나절 정도 쉬게 해주는 게 좋아요.
────────────────────────────
테스트 문장: 음식은 1주일 정도 유동식으로 드십시오.
리라이팅: 자기 몸 상태에 맞게 음식을 조절하세요.
────────────────────────────
테스트 문장: 덩어리 있는 것, 

In [36]:
import sys
sys.path.append('/content/drive/MyDrive')

from evaluate_utils import evaluate_model

evaluate_model([sentence], [rewrite], model_name="Ko-SRoBERTa Retrieval")


평가 결과 - Ko-SRoBERTa Retrieval
──────────────────────────────
KoBERTScore: 0.7372
평균 문장 길이: 5.00 단어
──────────────────────────────
